<a href="https://colab.research.google.com/github/s21sm/Python/blob/master/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# To import needed dependencies 

import io
from scipy.spatial import distance
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.spatial import distance

In [0]:
# upload files

uploaded = files.upload()

In [0]:
# if needed the file can be collected from

train_rss = np.genfromtxt ('Training_rss_21Aug17.csv', delimiter=",")
test_rss = np.genfromtxt ('Test_rss_21Aug17.csv', delimiter=",")
train_coordinate = np.genfromtxt ('Training_coordinates_21Aug17.csv', delimiter=",")
test_coordinate = np.genfromtxt ('Test_coordinates_21Aug17.csv', delimiter=",")

In [0]:
# For training data : changing the initial RSS value 100 to -103
for i in range(train_rss.shape[0]): 
 for j in range(train_rss.shape[1]):
  if train_rss[i,j] == 100:
    train_rss[i,j] = -103

# scaling into 0 to 1
for i in range(train_rss.shape[0]): 
 for j in range(train_rss.shape[1]):
   train_rss[i,j] = (train_rss[i,j] + 103) / 103

# For test data : changing the initial RSS value 100 to -103
for i in range(test_rss.shape[0]): 
 for j in range(test_rss.shape[1]):
  if test_rss[i,j] == 100:
    test_rss[i,j] = -103

# scaling into 0 to 1
for i in range(test_rss.shape[0]): 
 for j in range(test_rss.shape[1]):
   test_rss[i,j] = (test_rss[i,j] + 103) / 103

In [0]:
# Defining data for CNN model

(x_train, y_train), (x_test) = (train_rss, np.arange(697) ), (test_rss)
 

In [0]:
# no need scaling, manual scalling provides better result
x_train = tf.keras.utils.normalize (x_train, axis=1)
x_test = tf.keras.utils.normalize (x_test, axis=1)

In [0]:
# Modeling

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(512,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(697,activation=tf.nn.softmax))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x_train,y_train,epochs=20)

In [0]:
# prediction

prediction = model.predict([x_test])

In [0]:
# calculating mean error

dis = 0
for rr in range(x_test.shape[0]):
  estimated = train_coordinate[np.argmax(prediction[rr])]
  real = test_coordinate[rr]
  d = distance.euclidean(estimated, real)
  dis = dis+d

print(dis/x_test.shape[0])
